In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [23]:
# Load the models
model=load_model('model.h5')


#laod the encoder and scaler
with open('label_encoded_gender.pkl','rb') as files:
    label_encoder_gender=pickle.load(files)

with open('geo_one_hot_encoder.pkl','rb') as files:
    one_hot_encoder_geo=pickle.load(files)

with open('scaler.pkl','rb') as files:
    scaler=pickle.load(files) 

In [24]:
#example input

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [25]:
geo_encoded=one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

C:\Users\PINKY\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [26]:
input_df=pd.DataFrame([input_data])

In [27]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [28]:
# EWncode the categorical variable

input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])

In [29]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [30]:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [31]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [32]:
#Scaling the input data

input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [33]:
#Prediction

prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 133ms/step


array([[0.0564701]], dtype=float32)

In [34]:
prediction_prob=prediction[0][0]
prediction_prob

0.056470096

In [37]:
if prediction_prob>0.5:
    print("This custoner is likely to churn!")
else:
    print("This customer is not likely to churn!")

This customer is not likely to churn!
